# Creates links.txt file, which contains links to all the poems

In [1]:
links_concrete = {
    "contemporary": "https://www.poema.art.pl/kontener/24-polska-wspolczesna",
    "classics": "https://www.poema.art.pl/kontener/1-poezja-polska",
    }

BASE_ADDR = "https://www.poema.art.pl"

In [2]:
# structure: 
# links_dir -> first_letter -> name -> list of poems
links_dir = "https://www.poema.art.pl/kontener/35-debiuty-poezja"

In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
def fetch_conteners_list(link, with_titles=False):
    bs = BeautifulSoup(requests.get(link).content, "html.parser")
    links = []
    descs = []
    content = bs.find("div", "content")
    try:
        for c in content.findAll("p", "title"):
            x = c.find("a")
            links.append(x.attrs["href"])
            descs.append(x.string)
        links = [BASE_ADDR + l for l in links]
    except:
        pass
    
    if with_titles:
        return list(zip(links, descs))
    else:
        return links

def flatten(l):
    return [x for ll in l for x in ll]

In [5]:
from tqdm import tqdm
from time import sleep

In [6]:
links_amateur = []
for list_of_poems in tqdm(fetch_conteners_list(links_dir)):
    links_amateur.append(fetch_conteners_list(list_of_poems, with_titles=True))
    sleep(0.5)

100%|██████████| 33/33 [03:23<00:00,  2.82s/it]


In [7]:
links_amateur = flatten(links_amateur)

In [8]:
links_amateur[1000]

('https://www.poema.art.pl/kontener/13113-keyli-15', 'Keyli_15')

In [9]:
# some of the links are already "publications", others are "containers"
# we now have to split the list into containers and publications

In [10]:
publications = [(k, v) for (k, v) in links_amateur if "publikacja" in k]
containers = [(k, v) for (k, v) in links_amateur if "kontener" in k]

In [12]:
len(publications)

223

In [ ]:
links_amateur_by_author = {}
broken = []
for (link, author) in tqdm(containers):
    try:
        links_amateur_by_author[author] = fetch_conteners_list(link, with_titles=True)
    except:
        broken.append((link, author))
        print("Broken: {} : {}".format(link, author))
    sleep(0.1)


 26%|██▋       | 2480/9440 [1:10:26<2:12:29,  1.14s/it] 

Broken: https://www.poema.art.pl/kontener/15440-biondo : Biondo


 26%|██▋       | 2483/9440 [1:10:30<2:30:27,  1.30s/it]

Broken: https://www.poema.art.pl/kontener/6084-birek : Birek


 26%|██▋       | 2485/9440 [1:10:31<2:01:09,  1.05s/it]

...

 26%|██▋       | 2486/9440 [1:10:32<1:36:38,  1.20it/s]

Broken: https://www.poema.art.pl/kontener/19940-bitta-malina : Bitta Malina


 26%|██▋       | 2488/9440 [1:10:32<1:04:25,  1.80it/s]

Broken: https://www.poema.art.pl/kontener/11071-bitter-fraise : Bitter-fraise
Broken: https://www.poema.art.pl/kontener/16266-bittersweet : Bittersweet


 26%|██▋       | 2489/9440 [1:10:35<2:03:49,  1.07s/it]

In [49]:
with open("links_amateur.txt", "w") as f:
    f.write("\n".join(flatten(links_amateur)))

In [56]:
links_professional = []
for k, v in tqdm(links_concrete.items()):
    for l in fetch_conteners_list(v):
        links_professional.append(fetch_conteners_list(l))
        sleep(1)

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [54]:
links_professional[2]

'https://www.poema.art.pl/kontener/575-belwit-christian'

In [84]:
links = flatten(links)

In [94]:
all_poems_links = flatten(all_poems_links)

In [95]:
len(all_poems_links)

100535

In [104]:
with open("links.txt", "w") as f:
    for k, v in all_poems_links:
        try:
            f.write(k + "\t" + v + "\n")
        except:
            pass